In [31]:
import pandas as pd
import json

file_path_nechi = r'C:/Users/Mateo Grisales/Documents/NUVU/Caudal_Medio_Dia_Historico/LA ESPERANZA NECHI.csv'
file_path_la_raya = r'C:/Users/Mateo Grisales/Documents/NUVU/Caudal_Medio_Dia_Historico/LA RAYA.csv'
file_path_las_varas = r'C:/Users/Mateo Grisales/Documents/NUVU/Caudal_Medio_Dia_Historico/LAS VARAS.csv'
file_path_magangue = r'C:/Users/Mateo Grisales/Documents/NUVU/Caudal_Medio_Dia_Historico/MAGANGUE.csv'
file_path_sucre = r'C:/Users/Mateo Grisales/Documents/NUVU/Caudal_Medio_Dia_Historico/SUCRE.csv'
file_path_tres_cruces = r'C:/Users/Mateo Grisales/Documents/NUVU/Caudal_Medio_Dia_Historico/TRES CRUCES.csv'
file_path_villa_teresa = r'C:/Users/Mateo Grisales/Documents/NUVU/Caudal_Medio_Dia_Historico/VILLA TERESA.csv'

data_nechi = pd.read_csv(file_path_nechi, sep=';', encoding='ISO-8859-1')
data_san_jacinto = pd.read_csv(file_path_la_raya, sep=';', encoding='ISO-8859-1')
data_las_varas = pd.read_csv(file_path_las_varas, sep=';', encoding='ISO-8859-1')
data_magangue = pd.read_csv(file_path_magangue, sep=';', encoding='ISO-8859-1')
data_sucre = pd.read_csv(file_path_sucre, sep=';', encoding='ISO-8859-1')
data_tres_cruces = pd.read_csv(file_path_tres_cruces, sep=';', encoding='ISO-8859-1')
data_villa_teresa = pd.read_csv(file_path_villa_teresa, sep=';', encoding='ISO-8859-1')


In [32]:
def transform_data(data, lat, lon):
    melted_data = data.melt(id_vars=["year", "month"], var_name="day", value_name="Caudal")
    melted_data.dropna(subset=["Caudal"], inplace=True)
    melted_data['day'] = melted_data['day'].astype(int)
    melted_data['Fecha'] = pd.to_datetime(melted_data[['year', 'month', 'day']]) + pd.to_timedelta('00:00:00')
    melted_data.sort_values('Fecha', inplace=True)
    #melted_data['Fecha'] = melted_data['Fecha'].astype(str)
    melted_data['Latitud'] = lat
    melted_data['Longitud'] = lon
    return melted_data

data_nechi_transformed = transform_data(data_nechi, 8.030111111, -74.78555556)
data_san_jacinto_transformed = transform_data(data_san_jacinto, 8.344555556, -74.56122222)
data_las_varas_transformed = transform_data(data_las_varas, 8.39, -74.56)
data_magangue_transformed = transform_data(data_magangue, 9.24, -74.74166667)
data_sucre_transformed = transform_data(data_sucre, 8.809722222, -74.72222222)
data_tres_cruces_transformed = transform_data(data_tres_cruces, 8.703083333, -74.51730556)
data_villa_teresa_transformed = transform_data(data_villa_teresa, 9.023972222, -74.97666667)

In [33]:
combined_data = pd.concat([data_nechi_transformed, data_san_jacinto_transformed, data_las_varas_transformed, 
                           data_magangue_transformed,data_sucre_transformed, data_tres_cruces_transformed,
                           data_villa_teresa_transformed], ignore_index=True)

In [34]:
def df_to_geojson(df, properties, lat='Latitud', lon='Longitud'):
    geojson = {'type': 'FeatureCollection', 'features': []}
    for _, row in df.iterrows():
        feature = {
            'type': 'Feature',
            'properties': {},
            'geometry': {
                'type': 'Point',
                'coordinates': [row[lon], row[lat]]
            }
        }
        for prop in properties:
            if isinstance(row[prop], pd.Timestamp):
                feature['properties'][prop] = row[prop].isoformat()
            else:
                feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [35]:
properties = ['Fecha', 'Caudal']
geojson_data = df_to_geojson(combined_data, properties)

geojson_path = 'C:/Users/Mateo Grisales/Documents/NUVU/Caudales.geojson'
with open(geojson_path, 'w') as f:
    json.dump(geojson_data, f)

print(f"GeoJSON file created at {geojson_path}")


GeoJSON file created at C:/Users/Mateo Grisales/Documents/NUVU/Caudales.geojson
